In [ ]:
# Data manipulation
import numpy as np
import pandas as pd
import datetime

# To calculate Greeks
import mibian

def read_data(inst_name):
    opt = pd.read_csv(inst_name  + ".csv")            
    opt = opt[(opt['Option Type'] == 'CE') & (opt['Strike Price'] == 10000)]        
    opt = opt[['Symbol','Date','Expiry','Option Type','Strike Price','Close','futures_price']]         
    return opt

Nifty_Opt = read_data('../data_modules/NIFTY_GS_data')
Nifty_Opt = Nifty_Opt[:2]
Nifty_Opt.head()

Nifty_Opt.Expiry = pd.to_datetime(Nifty_Opt.Expiry)
Nifty_Opt.Date = pd.to_datetime(Nifty_Opt.Date)
Nifty_Opt['time_to_expiry']= (Nifty_Opt.Expiry - Nifty_Opt.Date).dt.days            
Nifty_Opt.head()    

def implied_volatility_options(opt):
    opt['IV'] = np.nan        
    for i in range(0,len(opt)):                 
        opt.iloc[i,opt.columns.get_loc('IV')] = mibian.BS([opt.iloc[i]['futures_price'], 
                                                               opt.iloc[i]['Strike Price'], 
                                                               0, 
                                                               opt.iloc[i]['time_to_expiry']], 
                                                              callPrice=opt.iloc[i]['Close']
                                                             ).impliedVolatility        
    return opt  

Nifty_Opt = implied_volatility_options(Nifty_Opt)    
Nifty_Opt.head()

def delta_options(opt):
    opt['delta'] = np.nan    
    for i in range(0,len(opt)):         
        if opt.iloc[i]['Option Type'] == 'CE':            
            opt.iloc[i,opt.columns.get_loc('delta')] = mibian.BS([opt.iloc[i]['futures_price'], 
                                                               opt.iloc[i]['Strike Price'], 
                                                               0, 
                                                               opt.iloc[i]['time_to_expiry']], 
                                                              volatility=opt.iloc[i]['IV']
                                                             ).callDelta        
    return opt  

Nifty_Opt = delta_options(Nifty_Opt)    
Nifty_Opt

Nifty_Lot_Size = 75
Nifty_Opt['total_delta'] = Nifty_Opt['delta'] * Nifty_Lot_Size
Nifty_Opt


Nifty_Opt['Fut_Quantity'] = -Nifty_Opt.total_delta // 5 * 5
Nifty_Opt

futures_pnl = (Nifty_Opt.futures_price - Nifty_Opt.futures_price.shift(1)) * Nifty_Opt.Fut_Quantity.shift(1)
futures_pnl


call_pnl = (Nifty_Opt.Close - Nifty_Opt.Close.shift(1))  * Nifty_Lot_Size
call_pnl

call_pnl + futures_pnl





ModuleNotFoundError: No module named 'numpy'